In [1]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 123kB/s 
     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 6.7MB 39.8MB/s 
     |████████████████████████████████| 983kB 41.4MB/s 
     |████████████████████████████████| 245kB 43.0MB/s 
     |████████████████████████████████| 675kB 37.7MB/s 
     |████████████████████████████████| 471kB 25.2MB/s 
     |████████████████████████████████| 3.9MB 38.1MB/s 
     |████████████████████████████████| 450kB 38.3MB/s 
     |████████████████████████████████| 890kB 35.1MB/s 
     |████████████████████████████████| 1.1MB 38.1MB/s 
     |████████████████████████████████| 3.8MB 42.5MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
train_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng1train.csv") #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng1test.csv") #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_1'],test_size=0.15,random_state=42) #training and validation split
target_names=[0,1] #names of classes

In [4]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [5]:
import ktrain
from ktrain import text
MODEL_NAME = 'xlnet-base-cased' #xlnet predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for xlnet
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get xlnet classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for xlnet

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 24
	95percentile : 46
	99percentile : 52


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 23
	95percentile : 43
	99percentile : 50


In [6]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 829 steps, validate for 28 steps
Epoch 1/4
829/829 [==============================] - 2537s 3s/step - loss: 0.6589 - accuracy: 0.6096 - val_loss: 0.6587 - val_accuracy: 0.6617
Epoch 2/4
829/829 [==============================] - 2514s 3s/step - loss: 0.6287 - accuracy: 0.6628 - val_loss: 0.6612 - val_accuracy: 0.6241
Epoch 3/4
829/829 [==============================] - 2517s 3s/step - loss: 0.6775 - accuracy: 0.5995 - val_loss: 0.6637 - val_accuracy: 0.6241
Epoch 4/4
829/829 [==============================] - 2521s 3s/step - loss: 0.6278 - accuracy: 0.6659 - val_loss: 0.6177 - val_accuracy: 0.6800


In [7]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.58      0.54      0.56       330
           1       0.73      0.76      0.75       548

    accuracy                           0.68       878
   macro avg       0.66      0.65      0.65       878
weighted avg       0.68      0.68      0.68       878



array([[178, 152],
       [129, 419]])

In [8]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  xlnet

In [9]:
test_b_text=list(test_b['text']) #get text for testing
test_b_label=list(test_b['task_1']) #get label of testing

In [10]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [11]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [12]:
from sklearn.metrics import classification_report

In [13]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.47      0.83      0.60       288
           1       0.92      0.68      0.79       865

    accuracy                           0.72      1153
   macro avg       0.70      0.76      0.69      1153
weighted avg       0.81      0.72      0.74      1153

